In [3]:
%pip install fastapi pydantic toml openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add round summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_66_4.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round =8
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


context: During the ride towards the self-storage facility, Professor Dagger tells you that he has a confession to make. You...
actions:
1. Listen to his confession
2. Ask him to keep what he has to say to himself
3. Threaten him that if he lied about the money, you would hurt him real bad


I select option 1. TURN: 1 


context: As the car hums through the foggy evening, Professor Dagger, with a tremble in his voice, confesses. He reveals the previous owner's deep involvement with the occult, mentioning a sinister ritual that led to a demonic possession within the family. The story sends chills down your spine, but it also piques your curiosity about the book's true value and power.
summary: Kyle chooses to listen to Professor Dagger's confession, learning about the occult background of the book's previous owner and the dark events that transpired. This revelation adds a layer of intrigue and danger to the quest for the book.

context: You and Professor Dagger begin scouring through the mess, leaving nothing unturned, yet, after two long hours, your hands hold nothing but dust. You sense the Professor’s eyes laced with disappointment, but your desire for a better life fuels your persistence to hunt the fourth book down. You...
actions:
1. Revisit the drawers for overlooked compartments
2. Rummage through the pile of mess once more
3. Look behind the large wooden shelf by the wall


I select option 3. TURN: 1 


context: As you approach the large wooden shelf, your fingers graze its ancient spine. With a firm grip, you attempt to move it, but it slips from your hands and collides with the wall, producing a hollow thud. Professor Dagger's eyes light up with sudden interest.
skill: Strength
difficulty: normal

I am making a skill check using STRENGTH against a difficulty of NORMAL.
And I rolled a 61 for a result of FAILURE. TURN: 2 


context: The shelf stubbornly refuses to budge further, mocking your efforts. Professor Dagger, not deterred by the failed attempt, steps forward. He places his hands against the false wall and begins to slide it across its surface, searching for something only he seems to understand.
actions:
1. Assist Professor Dagger in his search.
2. Inspect the area where the shelf hit the wall.
3. Suggest leaving the room to search elsewhere.

I select option 2. TURN: 3 


context: You kneel to inspect the area where the shelf collided with the wall, your eyes catching the subtle outlines of a hidden mechanism. Professor Dagger, noticing your focus, joins you. Together, you press against the outlines, and with a soft click, a section of the wall slides open, revealing a dark corridor effluviating a pungent scent.
summary: Kyle's persistence leads to the discovery of a hidden corridor behind a false wall, after a failed attempt to move a large wooden shelf. Professor Dagger's expertise and Kyle's keen observation unveil a new path laden with mystery and potential danger.

Context: You follow Professor Dagger down the malodorous corridor behind the false wall, leading you to a door holding back the evil within.
Before entering, Professor Dagger warns you not to look or speak to whatever is inside, for you may enter a state of frenzy. With that in mind, you…
actions:
1. Follow him closely in silence
2. Ask him what the foul odour is
3. Comment on the surroundings
